# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 '

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwar

In [13]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [14]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'portfolio page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'portfolio page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'patent',
   'url': 'https://patents.google.com/patent/US20210049536A1/'},
  {'type': 'partner page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'linkedin profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/09/15/ai-in-production-gen_ai-and-agentic_ai-on-aws-at-scale/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/'

In [15]:
select_relevant_links("https://huggingface.co")

{'links': [{'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'learn', 'url': 'https://huggingface.co/learn'},
  {'type': 'docs', 'url': 'https://huggingface.co/docs'},
  {'type': 'community forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'}]}

In [16]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [17]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 9 relevant links


{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'projects / portfolio',
   'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'projects / portfolio',
   'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'professional profile',
   'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'blog / insights', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'patent',
   'url': 'https://patents.google.com/patent/US20210049536A1/'},
  {'type': 'social media', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'social media',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [18]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 10 relevant links


{'links': [{'type': 'company homepage', 'url': 'https://huggingface.co/'},
  {'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'Blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'Docs hub', 'url': 'https://huggingface.co/docs'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [19]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [20]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 7 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
MiniMaxAI/MiniMax-M2
Updated
7 days ago
•
810k
•
1.05k
moonshotai/Kimi-Linear-48B-A3B-Instruct
Updated
4 days ago
•
19.4k
•
350
deepseek-ai/DeepSeek-OCR
Updated
1 day ago
•
2.25M
•
2.46k
dx8152/Qwen-Edit-2509-Multiple-angles
Updated
about 17 hours ago
•
224
briaai/FIBO
Updated
3 days ago
•
3.03k
•
222
Browse 1M+ models
Spaces
Running
on
CPU Upgrade
1.4k
1.4k
The Smol Training Playbook: The Secrets to Building World-Class LLMs
📝
Running
15.7k
15.7k
DeepSite v3
🐳
Generate any application by Vibe Coding
Running
2.27k
2.27k
Wan2.2 Animate

In [28]:
# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""


In [22]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [23]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 10 relevant links


"\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nMiniMaxAI/MiniMax-M2\nUpdated\n7 days ago\n•\n810k\n•\n1.05k\nmoonshotai/Kimi-Linear-48B-A3B-Instruct\nUpdated\n4 days ago\n•\n19.4k\n•\n350\ndeepseek-ai/DeepSeek-OCR\nUpdated\n1 day ago\n•\n2.25M\n•\n2.46k\ndx8152/Qwen-Edit-2509-Multiple-angles\nUpdated\nabout 17 hours ago\n•\n224\nbriaai/FIBO\nUpdated\n3 days ago\n•\n3.03k\n•\n222\nBrowse 1M+ models\nSpaces\nRunning\non\nCPU Upgrade\n1.4k

In [24]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [25]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 12 relevant links


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


# Hugging Face Brochure

---

## About Hugging Face

Hugging Face is the AI community building the future — a collaborative platform where the machine learning (ML) community comes together to create, share, discover, and advance models, datasets, and applications. Positioned as the home of machine learning, Hugging Face empowers engineers, scientists, and enthusiasts to innovate openly and ethically in AI through a high-impact open-source ecosystem.

---

## What We Offer

- **Models**: Access and contribute to over 1 million pre-trained models spanning varieties of ML tasks and modalities including text, image, video, audio, and 3D.
- **Datasets**: Explore 250,000+ datasets for training, evaluation, and experimentation from diverse domains.
- **Spaces**: Run and share AI apps built by the community to demonstrate model capabilities and use cases.
- **Community & Collaboration**: Join a vibrant network of ML practitioners sharing best practices, tutorials, research, and tools.
- **Open Source Stack**: Build faster and better using Hugging Face's widely recognized open-source ML libraries and tools.
- **Enterprise Solutions**: Access paid Compute and Enterprise products designed for teams and organizations aiming to scale AI development with advanced platform capabilities.

---

## Our Culture

At Hugging Face, community and collaboration are at the core. The company fosters an inclusive and ethically driven environment where sharing knowledge and open innovation pave the way toward building trustworthy AI for the future. Contributors span beginners to world-class experts who build together openly and responsibly, promoting transparency in machine learning advancements.

---

## Customers & Users

Hugging Face serves a broad audience across industries including technology, automotive, healthcare, education, and research. Customers range from:

- Individual ML engineers and scientists
- Academic researchers and students building AI portfolios
- Innovators in startups and tech companies leveraging AI models and datasets
- Large enterprises seeking scalable AI infrastructure and collaboration tools

The platform features trending models updated regularly and hosts thousands of AI applications accessible directly through Spaces, making cutting-edge AI research and development broadly accessible.

---

## Careers at Hugging Face

Hugging Face is continually growing and seeks talented individuals passionate about pushing the boundaries of AI and machine learning. The company values:

- Innovation and creativity in ML research and engineering
- Collaborative spirit and open-source ethos
- Commitment to ethical AI development and community building

Whether your background is research, engineering, product, or community engagement, Hugging Face provides an inspiring workplace to contribute to groundbreaking projects shaping the future of AI.

---

## Join Us

- Explore and contribute to a vast ecosystem of ML resources: models, datasets, apps
- Share your ML projects and build your professional profile
- Accelerate your AI development with scalable enterprise solutions
- Engage with a fast-growing global community committed to open and ethical AI

**Website:** [huggingface.co](https://huggingface.co)  
**Sign Up:** Create your free account and start collaborating today!

---

## Brand Highlights

- Recognizable logo symbols and vibrant brand colors (#FFD21E, #FF9D00, #6B7280)
- Positioned as the central hub for open-source machine learning innovations
- Emphasizes an ethical, transparent, and community-driven AI ecosystem

---

Unlock the power of collaborative AI with Hugging Face — where the future of machine learning is built together.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [26]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [27]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 11 relevant links


# Hugging Face Brochure

---

## Who We Are  
Hugging Face is *the AI community building the future*. We provide a collaborative platform where machine learning practitioners, researchers, and enterprises come together to create, share, and improve models, datasets, and applications. Our mission is to accelerate the development and democratization of AI technologies by fostering an open and vibrant community.

---

## Our Platform  
Hugging Face is home to:

- **1 Million+ Machine Learning Models**: Explore state-of-the-art models across modalities including text, image, video, audio, and even 3D.
- **250,000+ Datasets**: Access diverse and up-to-date datasets such as autonomous vehicle data, OCR datasets, prompts, and more.
- **400,000+ AI Applications (Spaces)**: Run and build applications directly on the platform, featuring everything from text generation to video animation.

Key features:  
- Host and collaborate on unlimited public machine learning models and datasets.  
- Build your AI portfolio by sharing your work with the global ML community.  
- Utilize the Hugging Face open-source stack for faster and easier development.  

---

## For Enterprises  
Hugging Face offers **Team & Enterprise solutions** designed for scalability, security, and compliance:

- **Advanced AI platform** with enterprise-grade security and access controls.  
- **Single Sign-On (SSO)** integration for secure identity management.  
- **Audit Logs** to monitor and control actions within your organization.  
- **Flexible contract options and dedicated support** to help your team accelerate AI projects.  

Pricing starts at $20/user/month for Team subscriptions, with customizable enterprise plans available.

---

## Our Customers  
Our platform serves a diverse user base ranging from individual machine learning practitioners and researchers to startups and large enterprises. Industries benefiting from Hugging Face include autonomous vehicles, natural language processing, computer vision, and multimedia content generation, among others.

---

## Company Culture  
At Hugging Face, we foster an **open, collaborative, and community-driven culture**. We believe in democratizing AI technology by supporting openness, accessibility, and shared knowledge. Whether you’re an enthusiastic ML developer, an academic researcher, or an enterprise AI team, you’ll find a welcoming environment to innovate, learn, and grow.

---

## Careers & Opportunities  
Join a fast-growing AI company at the forefront of the machine learning revolution! We look for talented individuals passionate about AI, open source, and community building. Opportunities exist across engineering, research, support, and enterprise solutions.

- Work with cutting-edge AI models and open-source technology.  
- Collaborate with an international community shaping the future of AI.  
- Contribute to tools and resources used by millions worldwide.  

Visit our website and sign up to explore career openings and join the AI community building the future.

---

## Get Started with Hugging Face  
Explore models, datasets, and AI apps: **https://huggingface.co**  
Sign up for free, collaborate with the global ML community, and accelerate your AI projects today!

---

*Hugging Face — The home of machine learning.*  
Build, share, and innovate together.

In [29]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 14 relevant links


# Welcome to Hugging Face: The AI Community Building the Future 🚀🤗

---

## Who Are We?  
Think of Hugging Face as the *friendly neighborhood* of the AI world—a buzzing collaboration platform where machine learning enthusiasts, pros, and curious minds unite to create, share, and innovate with *over one million* models, datasets, and applications. Whether you're an AI whisperer or just AI-curious, Hugging Face is your hub for exploration and connection.

---

## What’s the Big Deal?  
- **1M+ Models**: Dive into a treasure trove of pre-trained models ranging from text wizards to image masters, video magicians, and even 3D conjurers.  
- **250K+ Datasets**: Fuel your AI with diverse and rich data, from autonomous vehicle models to ChatGPT prompt collections.  
- **400K+ Applications ("Spaces")**: Play with cutting-edge AI apps, from video generation to OCR (Optical Character Recognition), all running live on the platform.  

In short: If AI is a party, Hugging Face is the main event where the coolest brains hang out and build the next-gen tech.

---

## Why Hugging Face?  
- **Open & Ethical AI**: We're not just about fancy algorithms—we're about building a future where AI is transparent, accessible, and *friendly*.  
- **Community-Powered Innovation**: Our vibrant community of over *millions* contributes, tests, and improves models daily. Your work gets exposure, feedback, and collaboration opportunities.  
- **Freedom to Explore Every Modality**: Text, image, video, audio, 3D... Why settle for one when you can master them all in one place?  
- **Build Your Portfolio**: Share your latest breakthroughs, get recognized, and become an ML rockstar.  
- **Enterprise Ready**: Need heavy-duty compute or dedicated team solutions? We've got scalable, paid options that fit startups to enterprises.  

---

## The Hugging Face Culture: 🤝 + 💡 = ❤️  
We’re a user-first, community-driven team that embodies:  
- **Collaboration Over Competition**: Sharing is caring, especially when you’re building the future of AI.  
- **Openness and Transparency**: Open source isn’t a buzzword; it’s our way of doing things.  
- **Learning and Growing Together**: Whether you’re new to ML or a seasoned engineer, there’s always room to learn—and teach—here.  
- **Fun Meets Function**: Because machine learning can be *seriously* complex, we like to keep the vibe supportive and entertained. Expect puns, playful mascots, and a warm welcome.

---

## Careers and Joining the Party 🎉  
Fancy yourself a machine learning engineer, data scientist, developer advocate, or the next AI visionary? Hugging Face is hiring!  
- Work alongside AI experts shaping the future of open-source machine learning.  
- Be part of a supportive, global community that values creativity and innovation.  
- Enjoy competitive benefits and the chance to make a tangible impact on how AI evolves.

*Trust us — the robots will thank you.*  

---

## For Customers & Investors  
Whether you want to tap into the fastest growing community-verified models or leverage enterprise-grade AI infrastructure, Hugging Face has you covered. We provide trusted compute environments and collaboration tools that accelerate machine learning innovation at scale.

---

## Ready to Hug the Future? 🤗  
Jump in at [huggingface.co](https://huggingface.co)  
- Explore AI apps  
- Browse 1 million+ models  
- Share your own creations  
- Join a community building a **brighter, friendlier AI world**

*Hugging Face: Because the future isn’t just smart—it's social.*

---

*P.S. Our logo is yellow and friendly, but our AI models are sharp as a tack. 😉*  
Colors? #FFD21E (sunshine yellow), #FF9D00 (orange zest), #6B7280 (cool slate) — bright, bold, and approachable, just like us!  

---

**Hugging Face – The AI community building the future, one smile (and one model) at a time.**

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>